# Processamento e Tratamento de Dados

## Etapa 1: Exame Nacional do Ensino Médio (ENEM)

Nesta primeira etapa, foi realizada a leitura e pré-processamento dos microdados do ENEM 2018.

In [1]:
# Importando bibliotecas e inicializando o Spark
import findspark
findspark.init("c:/spark")
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import *
from database_utils import GetIdeb, PrepareIdeb, GetMerged

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Criando a Sessão Spark para leitura dos dados
spark = SparkSession.builder \
        .master('local') \
        .appName('Leitura de dados do Enem') \
        .getOrCreate()

In [3]:
# Verificando informações da sessão que foi inicializada
spark

In [4]:
# Leitura do microdado
df = spark.read \
    .format('csv') \
    .option('header', 'true') \
    .option('sep', ';') \
    .load("C:\\Users\\barba\\Desktop\\MyMachineLearningProject\\MICRODADOS_ENEM_2018.csv" )

In [5]:
# Verificando o total de registros 
df.count()

5513747

In [6]:
# Selecionando o universo desejado pela situação de conclusão do EM

# Selecionando todos os alunos que estão cursando o terceiro ano do EM no ano de 2018
df = df.filter(df.TP_ST_CONCLUSAO == '2')
df = df.filter(df.Q026 == 'B')

# Selecionando somente os alunos que estiveram presentes em todas as provas 
df = df.filter((df.TP_PRESENCA_CN == '1') & (df.TP_PRESENCA_CH == '1') & (df.TP_PRESENCA_LC == '1') & (df.TP_PRESENCA_MT == '1'))

# Eliminando os alunos que deixaram a redação em branco
df = df.filter(df.TP_STATUS_REDACAO != '4')

In [7]:
# Verificando o total de registros 
df.count()

1256575

In [8]:
# Selecionando somente os alunos que apresentavam o campo de identificação da escola em que estudavam preenchido
df = df.filter(df.CO_ESCOLA.isNotNull())

In [9]:
# Verificando o total de registros após seleção do universo
df.count()

1119954

In [10]:
# Cálculo da Nota total do ENEM (soma das 4 notas + redação)
df = df.withColumn('NU_NOTA_TOT', 
                   col('NU_NOTA_CN') + col('NU_NOTA_CH') + col('NU_NOTA_LC') + col('NU_NOTA_MT') + col('NU_NOTA_REDACAO'))

In [11]:
# Eliminando notas componentes da variável resposta do modelo
df = df.drop('NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO')

# Eliminando variáveis relacionadas ao participante
drop_list = ['NU_INSCRICAO',
             'NU_ANO', 
             'CO_MUNICIPIO_RESIDENCIA',
             'NO_MUNICIPIO_RESIDENCIA', 
             'CO_UF_RESIDENCIA',
             'CO_MUNICIPIO_NASCIMENTO', 
             'NO_MUNICIPIO_NASCIMENTO', 
             'CO_UF_NASCIMENTO',
             'SG_UF_NASCIMENTO',
             'TP_ST_CONCLUSAO', 
             'TP_ANO_CONCLUIU', 
             'IN_TREINEIRO']

df = df.drop(*drop_list)

# Eliminando variáveis relacionadas à escola
df = df.drop('NO_MUNICIPIO_ESC', 'CO_MUNICIPIO_ESC', 'CO_UF_ESC', 'SG_UF_ESC')

# Eliminando variáveis relacionadas a pedidos de atendimento especializado
drop_list = ['IN_BAIXA_VISAO',
             'IN_CEGUEIRA',
             'IN_SURDEZ',
             'IN_DEFICIENCIA_AUDITIVA',
             'IN_SURDO_CEGUEIRA',
             'IN_DEFICIENCIA_FISICA',
             'IN_DEFICIENCIA_MENTAL',
             'IN_DEFICIT_ATENCAO',
             'IN_DISLEXIA',
             'IN_DISCALCULIA',
             'IN_AUTISMO',
             'IN_VISAO_MONOCULAR',
             'IN_OUTRA_DEF',
             'IN_GESTANTE',
             'IN_LACTANTE',
             'IN_IDOSO',
             'IN_ESTUDA_CLASSE_HOSPITALAR',
             'IN_SEM_RECURSO',
             'IN_BRAILLE',
             'IN_AMPLIADA_24',
             'IN_AMPLIADA_18',
             'IN_LEDOR',
             'IN_ACESSO',
             'IN_TRANSCRICAO',
             'IN_LIBRAS',
             'IN_LEITURA_LABIAL',
             'IN_MESA_CADEIRA_RODAS',
             'IN_MESA_CADEIRA_SEPARADA',
             'IN_APOIO_PERNA',
             'IN_GUIA_INTERPRETE',
             'IN_COMPUTADOR',
             'IN_CADEIRA_ESPECIAL',
             'IN_CADEIRA_CANHOTO',
             'IN_CADEIRA_ACOLCHOADA',
             'IN_PROVA_DEITADO',
             'IN_MOBILIARIO_OBESO',
             'IN_LAMINA_OVERLAY',
             'IN_PROTETOR_AURICULAR',
             'IN_MEDIDOR_GLICOSE',
             'IN_MAQUINA_BRAILE',
             'IN_SOROBAN',
             'IN_MARCA_PASSO',
             'IN_SONDA',
             'IN_MEDICAMENTOS',
             'IN_SALA_INDIVIDUAL',
             'IN_SALA_ESPECIAL',
             'IN_SALA_ACOMPANHANTE',
             'IN_MOBILIARIO_ESPECIFICO',
             'IN_MATERIAL_ESPECIFICO',
             'IN_NOME_SOCIAL']

df = df.drop(*drop_list)

# Eliminando variáveis relacionadas ao local de aplicação da prova
drop_list = ['CO_MUNICIPIO_PROVA',
             'NO_MUNICIPIO_PROVA',
             'CO_UF_PROVA',
             'SG_UF_PROVA']

df = df.drop(*drop_list)

# Eliminando variáveis relacionadas às provas objetivas
drop_list = ['TP_PRESENCA_CN',
             'TP_PRESENCA_CH',
             'TP_PRESENCA_LC',
             'TP_PRESENCA_MT',
             'CO_PROVA_CN',
             'CO_PROVA_CH',
             'CO_PROVA_LC',
             'CO_PROVA_MT',
             'TX_RESPOSTAS_CN',
             'TX_RESPOSTAS_CH',
             'TX_RESPOSTAS_LC',
             'TX_RESPOSTAS_MT',
             'TX_GABARITO_CN',
             'TX_GABARITO_CH',
             'TX_GABARITO_LC',
             'TX_GABARITO_MT']

df = df.drop(*drop_list)

# Eliminando variáveis relacionadas à redação
drop_list = ['NU_NOTA_COMP1',
             'NU_NOTA_COMP2',
             'NU_NOTA_COMP3',
             'NU_NOTA_COMP4',
             'NU_NOTA_COMP5',
             'TP_STATUS_REDACAO']

df = df.drop(*drop_list)

# Eliminando Q026 utilizada na seleção do universo
drop_list = ['Q026']
df = df.drop(*drop_list)

In [12]:
# Verificando variáveis que restaram 
df.printSchema()

root
 |-- SG_UF_RESIDENCIA: string (nullable = true)
 |-- NU_IDADE: string (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- TP_ESTADO_CIVIL: string (nullable = true)
 |-- TP_COR_RACA: string (nullable = true)
 |-- TP_NACIONALIDADE: string (nullable = true)
 |-- TP_ESCOLA: string (nullable = true)
 |-- TP_ENSINO: string (nullable = true)
 |-- CO_ESCOLA: string (nullable = true)
 |-- TP_DEPENDENCIA_ADM_ESC: string (nullable = true)
 |-- TP_LOCALIZACAO_ESC: string (nullable = true)
 |-- TP_SIT_FUNC_ESC: string (nullable = true)
 |-- TP_LINGUA: string (nullable = true)
 |-- Q001: string (nullable = true)
 |-- Q002: string (nullable = true)
 |-- Q003: string (nullable = true)
 |-- Q004: string (nullable = true)
 |-- Q005: string (nullable = true)
 |-- Q006: string (nullable = true)
 |-- Q007: string (nullable = true)
 |-- Q008: string (nullable = true)
 |-- Q009: string (nullable = true)
 |-- Q010: string (nullable = true)
 |-- Q011: string (nullable = true)
 |-- Q012: string (

In [14]:
# Transformando o DataFrame Spark em DataFrame Pandas
df_enem = df.select("*").toPandas()

# Finalizando a Sessão Spark
spark.stop()

## Etapa 2: Índice de Desenvolvimento da Educação Básica (IDEB)

Nesta etapa, foi realizada a leitura e pré-processamento dos dados do IDEB desagregado por escola.

In [21]:
# Leitura
df_ideb = GetIdeb('http://download.inep.gov.br/educacao_basica/portal_ideb/planilhas_para_download/2017/divulgacao_ensino_medio-escolas-2017.zip')

Download do arquivo IDEB concluído


In [22]:
df_ideb.head()

,Sigla da UF,Código do Município,Nome do Município,Código da Escola,Nome da Escola,Rede,Taxa de Aprovação - 2017,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Nota SAEB - 2017,Unnamed: 14,Unnamed: 15,IDEB 2017 (N x P),Projeções,Unnamed: 18
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matemática,Língua Portuguesa,Nota Média Padronizada (N),NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,Total,1ª série,2ª série,3ª série,4ª série,Não\nseriado***,Indicador de Rendimento (P),NaN,NaN,NaN,NaN,2019,2021
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RO,1100015.0,Alta Floresta D'Oeste,11024968.0,EEEFM JUSCELINO KUBITSCHEK DE OLIVEIRA,Estadual,96.1,96.2,95.1,96.4,-,-,0.958966,297.74,286.88,5.16587,5,5.2,5.4
4,RO,1100015.0,Alta Floresta D'Oeste,11025638.0,EEEFM PADRE EZEQUIEL RAMIN,Estadual,85.1,80.2,89,87.8,-,-,0.85484,255.04,247.86,3.98202,3.4,3.6,3.8


In [23]:
# Pré-processamento
df_ideb = PrepareIdeb(df_ideb)

Pré-processamento da base do IDEB concluído


In [24]:
df_ideb.head()

,CO_ESCOLA,IN_RENDIMENTO,NT_PADRONIZADA,IDEB
3,11024968,0.958966,5.165867,5.0
4,11025638,0.854840,3.982016,3.4
5,11007885,0.935347,5.251353,4.9
6,11006889,0.713505,4.362813,3.1
7,11050233,0.846576,NaN,NaN


## Etapa 3: Merge

Nesta etapa, foi realizado o merge entre os dataframes, algumas variáveis com alto percentual de dados faltantes foram eliminadas e foram aplicados métodos de imputação a outras.

In [25]:
df = GetMerged(enem = df_enem, ideb = df_ideb)

Gerador de massa de dados concluído com sucesso


In [26]:
df.head()

,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ESCOLA,TP_ENSINO,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,...,Q019,Q020,Q021,Q022,Q023,Q024,Q025,Q027,NU_NOTA_TOT,IN_RENDIMENTO
0,RO,17.0,M,0,1,1,4,1,4,1,...,D,B,A,D,B,B,B,B,2932.9,0.984189
1,RO,17.0,F,0,3,1,4,1,4,1,...,B,A,A,D,B,B,B,D,2790.0,0.984189
2,RO,17.0,F,0,1,1,4,1,4,1,...,C,A,B,C,A,C,B,D,3090.2,0.984189
3,RO,17.0,M,0,1,1,4,1,4,1,...,C,B,B,E,B,C,B,D,2908.3,0.984189
4,RO,18.0,M,0,3,1,4,1,4,1,...,B,B,A,E,B,C,B,E,3365.2,0.984189


In [27]:
# Verificando dimensões da base de dados tratada
df.shape

(1119954, 40)

In [28]:
# Exportando base tratada
df.to_csv('df_tratado.csv', decimal=',', sep=';', index=False)